In [1]:
import torch

# Check if CUDA (GPU) is available
if torch.cuda.is_available():
    # Get the number of available GPUs
    num_gpus = torch.cuda.device_count()
    
    # Print GPU information
    print(f"CUDA (GPU) is available with {num_gpus} GPU(s).")
    for gpu_id in range(num_gpus):
        print(f"GPU {gpu_id}: {torch.cuda.get_device_name(gpu_id)}")
else:
    print("CUDA (GPU) is not available. Training will be done on CPU.")


CUDA (GPU) is available with 2 GPU(s).
GPU 0: NVIDIA RTX A6000
GPU 1: NVIDIA RTX A6000


In [2]:
import torch
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
import pandas as pd

In [3]:
df = pd.read_csv('/home/asasmal/NCRI/Twitterdatainsheets.csv')

/tmp/ipykernel_872081/4166510820.py:1: DtypeWarning: Columns (3,4,5,6,12,14) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/home/asasmal/NCRI/Twitterdatainsheets.csv')


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 206295 entries, 0 to 206294
Data columns (total 15 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   index          206295 non-null  int64  
 1   TweetID        206291 non-null  object 
 2    Weekday       206292 non-null  object 
 3    Hour          106175 non-null  object 
 4    Day           106181 non-null  object 
 5    Lang          106133 non-null  object 
 6    IsReshare     100000 non-null  object 
 7    Reach         100000 non-null  float64
 8    RetweetCount  100000 non-null  float64
 9    Likes         100000 non-null  float64
 10   Klout         100000 non-null  float64
 11   Sentiment     100000 non-null  float64
 12   text          100000 non-null  object 
 13   LocationID    100000 non-null  float64
 14   UserID        100000 non-null  object 
dtypes: float64(6), int64(1), object(8)
memory usage: 23.6+ MB


In [5]:
column_names = df.columns

In [6]:
column_names

Index(['index', 'TweetID', ' Weekday', ' Hour', ' Day', ' Lang', ' IsReshare',
       ' Reach', ' RetweetCount', ' Likes', ' Klout', ' Sentiment', ' text',
       ' LocationID', ' UserID'],
      dtype='object')

In [7]:
sentiment_column =' Sentiment'

In [8]:
# Find the lowest and highest numbers in the column
lowest_number = df[sentiment_column].min()
highest_number = df[sentiment_column].max()

# Print the results
print(f"Lowest number in {sentiment_column}: {lowest_number}")
print(f"Highest number in {sentiment_column}: {highest_number}")


Lowest number in  Sentiment: -6.0
Highest number in  Sentiment: 7.333333


In [9]:
text =' text'

In [10]:
# Check for duplicates in the specified column
duplicates_mask = df[text].duplicated()

# Filter the DataFrame to get rows with duplicate values in the specified column
duplicate_rows = df[duplicates_mask]

# Print duplicate values if any
if not duplicate_rows.empty:
    print(f"Duplicates in {text} column:")
    print(duplicate_rows[text].unique())
else:
    print(f"No duplicates found in {text} column.")

Duplicates in  text column:
['RT @paf31: PureScript now runs in the browser on Node natively (C++11 backend) on iOS and Android (React Native) and on AWS Lambda! Did'
 'RT @jpaulreed: Happy New Year to all those AWS instances of ours!'
 'RT @linuxacademyCOM: AWS training library available with all three associate certifications and AWS CSA Professional starting at just $9! h'
 ...
 'RT @GNethercutt: Find me at #PearlHacks this weekend for help with your #AWS Java big data and cloud hacks!'
 'RT @kylemroche: Adding websockets to an @awscloud serverless app with #AWS #IoT https://stesie.github.io/2016/04/aws-iot-pubsub'
 nan]


In [11]:
df_no_duplicates = df.drop_duplicates(subset=[text])

In [12]:
df_no_duplicates

,index,TweetID,Weekday,Hour,Day,Lang,IsReshare,Reach,RetweetCount,Likes,Klout,Sentiment,text,LocationID,UserID
0,0,tw-682712873332805633,Thursday,17,31,en,False,44.0,0.0,0.0,35.0,0.0,We are hiring: Senior Software Engineer - Prot...,3751.0,tw-40932430
1,1,tw-682713045357998080,Thursday,17,31,en,True,1810.0,5.0,0.0,53.0,2.0,RT @CodeMineStatus: This is true Amazon Web Se...,3989.0,tw-3179389829
2,2,tw-682713219375476736,Thursday,17,31,en,False,282.0,0.0,0.0,47.0,0.0,Devops Engineer Aws Ansible Cassandra Mysql Ub...,3741.0,tw-4624808414
3,3,tw-682713436967579648,Thursday,17,31,en,False,2087.0,4.0,0.0,53.0,0.0,Happy New Year to all those AWS instances of o...,3753.0,tw-356447127
4,4,tw-682714048199311366,Thursday,17,31,en,False,953.0,0.0,0.0,47.0,0.0,Amazon is hiring! #Sr. #International Tax Mana...,3751.0,tw-3172686669
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99993,99993,tw-716349145771065344,Saturday,13,2,en,False,763.0,0.0,0.0,29.0,0.0,I added a video to a @YouTube playlist https:/...,1332.0,tw-1143330170
99995,99995,tw-716349486742700032,Saturday,13,2,hi,True,1143.0,1.0,0.0,49.0,1.0,RT @iiManjula: Dekhana thoda dhirese darlingme...,1349.0,tw-4667163978
99996,99996,tw-716349615340199937,Saturday,13,2,en,False,763.0,0.0,0.0,29.0,0.0,I added a video to a @YouTube playlist https:/...,1332.0,tw-1143330170
99999,99999,tw-716352771209887744,Saturday,13,2,en,False,2422.0,1.0,0.0,34.0,3.0,Great turn out for the AWS Plant Sale and the ...,4007.0,tw-2286856224


In [13]:

df_no_duplicates['overall_sentiment'] = (df_no_duplicates[sentiment_column] > 0).astype(int)


/tmp/ipykernel_872081/966774818.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_no_duplicates['overall_sentiment'] = (df_no_duplicates[sentiment_column] > 0).astype(int)


In [14]:
df_no_duplicates

,index,TweetID,Weekday,Hour,Day,Lang,IsReshare,Reach,RetweetCount,Likes,Klout,Sentiment,text,LocationID,UserID,overall_sentiment
0,0,tw-682712873332805633,Thursday,17,31,en,False,44.0,0.0,0.0,35.0,0.0,We are hiring: Senior Software Engineer - Prot...,3751.0,tw-40932430,0
1,1,tw-682713045357998080,Thursday,17,31,en,True,1810.0,5.0,0.0,53.0,2.0,RT @CodeMineStatus: This is true Amazon Web Se...,3989.0,tw-3179389829,1
2,2,tw-682713219375476736,Thursday,17,31,en,False,282.0,0.0,0.0,47.0,0.0,Devops Engineer Aws Ansible Cassandra Mysql Ub...,3741.0,tw-4624808414,0
3,3,tw-682713436967579648,Thursday,17,31,en,False,2087.0,4.0,0.0,53.0,0.0,Happy New Year to all those AWS instances of o...,3753.0,tw-356447127,0
4,4,tw-682714048199311366,Thursday,17,31,en,False,953.0,0.0,0.0,47.0,0.0,Amazon is hiring! #Sr. #International Tax Mana...,3751.0,tw-3172686669,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99993,99993,tw-716349145771065344,Saturday,13,2,en,False,763.0,0.0,0.0,29.0,0.0,I added a video to a @YouTube playlist https:/...,1332.0,tw-1143330170,0
99995,99995,tw-716349486742700032,Saturday,13,2,hi,True,1143.0,1.0,0.0,49.0,1.0,RT @iiManjula: Dekhana thoda dhirese darlingme...,1349.0,tw-4667163978,1
99996,99996,tw-716349615340199937,Saturday,13,2,en,False,763.0,0.0,0.0,29.0,0.0,I added a video to a @YouTube playlist https:/...,1332.0,tw-1143330170,0
99999,99999,tw-716352771209887744,Saturday,13,2,en,False,2422.0,1.0,0.0,34.0,3.0,Great turn out for the AWS Plant Sale and the ...,4007.0,tw-2286856224,1


In [15]:
column_names = df_no_duplicates.columns

In [16]:
column_names


Index(['index', 'TweetID', ' Weekday', ' Hour', ' Day', ' Lang', ' IsReshare',
       ' Reach', ' RetweetCount', ' Likes', ' Klout', ' Sentiment', ' text',
       ' LocationID', ' UserID', 'overall_sentiment'],
      dtype='object')

In [17]:
import pandas as pd

# Assuming df is your DataFrame
columns_to_keep = [' text', 'overall_sentiment']

# Keep only the specified columns
df_filtered = df_no_duplicates[columns_to_keep]


In [18]:
df_filtered

,text,overall_sentiment
0,We are hiring: Senior Software Engineer - Prot...,0
1,RT @CodeMineStatus: This is true Amazon Web Se...,1
2,Devops Engineer Aws Ansible Cassandra Mysql Ub...,0
3,Happy New Year to all those AWS instances of o...,0
4,Amazon is hiring! #Sr. #International Tax Mana...,0
...,...,...
99993,I added a video to a @YouTube playlist https:/...,0
99995,RT @iiManjula: Dekhana thoda dhirese darlingme...,1
99996,I added a video to a @YouTube playlist https:/...,0
99999,Great turn out for the AWS Plant Sale and the ...,1


In [19]:
df_filtered_no_null_columns = df_filtered.reset_index(drop=True)

In [20]:
df_filtered_no_null_columns

,text,overall_sentiment
0,We are hiring: Senior Software Engineer - Prot...,0
1,RT @CodeMineStatus: This is true Amazon Web Se...,1
2,Devops Engineer Aws Ansible Cassandra Mysql Ub...,0
3,Happy New Year to all those AWS instances of o...,0
4,Amazon is hiring! #Sr. #International Tax Mana...,0
...,...,...
70267,I added a video to a @YouTube playlist https:/...,0
70268,RT @iiManjula: Dekhana thoda dhirese darlingme...,1
70269,I added a video to a @YouTube playlist https:/...,0
70270,Great turn out for the AWS Plant Sale and the ...,1


In [21]:
value_counts = df_filtered_no_null_columns['overall_sentiment'].value_counts()

# Print the number of occurrences of 0 and 1
print(value_counts)

overall_sentiment
0    51436
1    18836
Name: count, dtype: int64


In [22]:
#output_file_path = 'Cleaned.csv'

# Save the DataFrame to a CSV file
#df_filtered_no_null_columns.to_csv(output_file_path, index=False)

In [8]:

import torch
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
import pandas as pd


In [9]:
#dataset = load_dataset("csv", data_files="Cleaned.csv",split='train[:10%]+train[-80%:]')

In [10]:
from datasets import load_dataset
ds = load_dataset('csv', data_files='Cleaned.csv')['train'].train_test_split(train_size=0.8, test_size=0.2)
print(ds)

DatasetDict({
    train: Dataset({
        features: ['text', 'labels'],
        num_rows: 19918
    })
    test: Dataset({
        features: ['text', 'labels'],
        num_rows: 4980
    })
})


In [11]:
num_classes = len(ds['train'].unique('labels'))

Flattening the indices:   0%|          | 0/19918 [00:00<?, ? examples/s]

In [12]:
num_classes

2

In [13]:
# Assuming columns are named 'text' and 'class'
train_data = ds['train']
test_data = ds['test']

# Step 2: Set up the model
model_name = 'bert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

# Step 3: Tokenize and Prepare Data
def tokenize_batch(batch):
    return tokenizer(batch['text'], padding=True, truncation=True, return_tensors='pt')

train_data = train_data.map(tokenize_batch, batched=True)

test_data = test_data.map(tokenize_batch, batched=True)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/19918 [00:00<?, ? examples/s]

Map:   0%|          | 0/4980 [00:00<?, ? examples/s]

In [14]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [21]:
from sklearn.metrics import f1_score

# Define a function to compute F1 score during evaluation
def compute_f1(predictions, labels):
    preds = predictions.argmax(axis=1)
    return f1_score(labels, preds, average='weighted')

In [22]:
# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
)

# Define Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=test_data,
    compute_metrics=lambda p: {'f1': compute_f1(p.predictions, p.label_ids)},
)


# Train the model
trainer.train()


Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


Step,Training Loss
500,0.002600
1000,0.017700
1500,0.015400
2000,0.006400
2500,0.002400
3000,0.001600
3500,0.001000


/home/asasmal/myenv/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
Checkpoint destination directory ./results/checkpoint-1000 already exists and is non-empty.Saving will proceed but saved results may be invalid.
/home/asasmal/myenv/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/home/asasmal/myenv/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/home/asa

TrainOutput(global_step=3735, training_loss=0.006322357415095709, metrics={'train_runtime': 895.3073, 'train_samples_per_second': 66.741, 'train_steps_per_second': 4.172, 'total_flos': 1.572193800198144e+16, 'train_loss': 0.006322357415095709, 'epoch': 3.0})

In [23]:
# Evaluate the model on the test set
results = trainer.evaluate()

# Print the evaluation results
print(results)

# Save the model with the best F1 score on the test set
trainer.save_model('./best_model')

{'eval_loss': 0.0034655514173209667, 'eval_f1': 0.9995983921491539, 'eval_runtime': 27.2891, 'eval_samples_per_second': 182.491, 'eval_steps_per_second': 11.433, 'epoch': 3.0}
